## A. Configuraciones generales.

In [ ]:
#1. Librerías.
%run "./librerias.ipynb"

In [ ]:
#2. Constantes.
#a. Constantes generales.
%run "./constantes.ipynb"

#b. Constantes a definir por el usuario.
#i. Semillerios a leer.
semillerio_1 = dataset_semillerio_all_1
semillerio_2 = dataset_semillerio_all_1_lagsdelta
#ii. Información sobre rango temporal del modelo.
cantidad_meses_train = "all"
ventana = 1

In [ ]:
#3. Lectura de datos.
#i. Semillerio modelo 1.
df_semillerio_1 = pd.read_csv(semillerio_1)
#ii. Semillerio modelo 2.
df_semillerio_2 = pd.read_csv(semillerio_2)

## B. Voting de promedios finales.

#### 1. Pre-procesamiento.

In [ ]:
#1. Calculamos la probabilidad promedio del semillerio 1 y filtramos por las columnas de interés.
df_semillerio_1_total = df_semillerio_1.copy()
df_semillerio_1_total["probabilidad_promedio"] = df_semillerio_1_total.filter(like="prob_semilla_").mean(axis=1)
resultados_promedio_1_total = df_semillerio_1_total[["numero_de_cliente","foto_mes","probabilidad_promedio"]]

In [ ]:
#2. Calculamos la probabilidad proemdio del semillerio 2 y filtramos por las columnas de interés.
df_semillerio_2_total = df_semillerio_2.copy()
df_semillerio_2_total["probabilidad_promedio"] = df_semillerio_2_total.filter(like="prob_semilla_").mean(axis=1)
resultados_promedio_2_total = df_semillerio_2_total[["numero_de_cliente","foto_mes","probabilidad_promedio"]]

In [ ]:
#3. Mergeamos ambos semilleríos en uno solo.
resultados_promedio_total = resultados_promedio_1_total.merge(resultados_promedio_2_total,how="left",on=["numero_de_cliente","foto_mes"])

In [ ]:
#4. Promedio ambos semillerios y me quedo con una probabilidad única por numero_de_cliente-foto_mes.
resultados_promedio_total["probabilidad_promedio"] = (resultados_promedio_total["probabilidad_promedio_x"] + resultados_promedio_total["probabilidad_promedio_y"])/2

In [ ]:
#5. Filtro las columnas de interés.
resultados_promedio_total = resultados_promedio_total[["numero_de_cliente","foto_mes","probabilidad_promedio"]]

#### 2. Subida a Kaggle.

In [ ]:
#a. Subimos a Kaggle el resutado ensamblado.
#i. Activamos la API.
api = KaggleApi()
api.authenticate()
#ii. Ordenamos a los clientes por probabilidad de ser "BAJA" de forma descendente.
tb_entrega = resultados_promedio_total.sort_values(by='probabilidad_promedio', ascending=False)
#iii. Genero una lista de distintos cortes candidatos, para enviar a Kaggle.
cortes = range(8500,13500,250)
#iv. Generamos las distintas predicciones de clases a partir de los distintos cortes posibles.
num_subida_kaggle = 1
print("--> Comienzan los envíos a Kaggle...\n")
for envios in cortes:
    #1. Le ponemos clase 1 ("BAJA") a los primeros "envios" con mayor probabilidad.
    tb_entrega['Predicted'] = 0
    tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
    resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
    print("Cantidad de clientes {}".format(envios))
    #2. Guardamos el archivo para Kaggle.
    nombre_archivo = "K_{}_{}_EK_0{}.csv".format(cantidad_meses_train,ventana,num_subida_kaggle) #DF con -X meses, Y ventana, número de intento.
    ruta_archivo= "{}/{}".format(exp_path,nombre_archivo)
    resultados.to_csv(ruta_archivo, index=False)
    
    num_subida_kaggle += 1
    
    #3. Envío a Kaggle.
    #a. Defino los parámetros claves.
    mensaje = f'Archivo {nombre_archivo}.Train {cantidad_meses_train} ensamblado de modelos. punto de corte:{envios}' # Datos de Train, si está con Data Drifting o no, semilla y punto de corte de prueba.
    competencia = 'dm-ey-f-2024-tercera'
    #c. Subo la Submission.
    api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
    print("Submission successful!")

## C. Voting de promedios parciales (numero_de_cliente-foto_mes).

#### 1. Preprocesamiento.

In [ ]:
#a. Concatenamos ambos dataframes.
df_semillerio_1_parcial = df_semillerio_1.copy()
df_semillerio_2_parcial = df_semillerio_2.copy()
df_semillerio_2_parcial.columns = df_semillerio_1_parcial.columns
df_semillerio_parcial = pd.concat([df_semillerio_1_parcial, df_semillerio_2_parcial], axis=0, ignore_index=True)

In [ ]:
#b. Agrupamos por numero_de_cliente, foto_mes, y sacamos el promedio por caso.
df_semillerio_parcial_agrupado = df_semillerio_parcial.groupby(["numero_de_cliente","foto_mes"]).mean().reset_index()

In [ ]:
#c. Ahora sí, calculamos el promedio general.
df_semillerio_parcial_agrupado["probabilidad_promedio"] = df_semillerio_parcial_agrupado.filter(like="prob_semilla_").mean(axis=1)

In [ ]:
#d. Nos quedamos con las columnas de interés.
df_semillerio_parcial_agrupado = df_semillerio_parcial_agrupado[["numero_de_cliente","foto_mes","probabilidad_promedio"]]

#### 2. Subida a Kaggle.

In [ ]:
#a. Subimos a Kaggle el resutado ensamblado.
#i. Activamos la API.
api = KaggleApi()
api.authenticate()
#ii. Ordenamos a los clientes por probabilidad de ser "BAJA" de forma descendente.
tb_entrega = df_semillerio_parcial_agrupado.sort_values(by='probabilidad_promedio', ascending=False)
#iii. Genero una lista de distintos cortes candidatos, para enviar a Kaggle.
cortes = range(8500,13500,250)
#iv. Generamos las distintas predicciones de clases a partir de los distintos cortes posibles.
num_subida_kaggle = 1
print("--> Comienzan los envíos a Kaggle...\n")
for envios in cortes:
    #1. Le ponemos clase 1 ("BAJA") a los primeros "envios" con mayor probabilidad.
    tb_entrega['Predicted'] = 0
    tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
    resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
    print("Cantidad de clientes {}".format(envios))
    #2. Guardamos el archivo para Kaggle.
    nombre_archivo = "K_votingparcial_{}_{}_EK_0{}.csv".format(cantidad_meses_train,ventana,num_subida_kaggle) #DF con -X meses, Y ventana, número de intento.
    ruta_archivo= "{}/{}".format(exp_path,nombre_archivo)
    resultados.to_csv(ruta_archivo, index=False)
    
    num_subida_kaggle += 1
    
    #3. Envío a Kaggle.
    #a. Defino los parámetros claves.
    mensaje = f'Archivo {nombre_archivo}.Train {cantidad_meses_train} ensamblado de modelos parciales. punto de corte:{envios}' # Datos de Train, si está con Data Drifting o no, semilla y punto de corte de prueba.
    competencia = 'dm-ey-f-2024-tercera'
    #c. Subo la Submission.
    api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
    print("Submission successful!")